In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam
from sklearn.metrics import confusion_matrix

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

#print("TensorFlow version: ", tf.__version__)
#assert version.parse(tf.__version__).release[0] >= 2, \
#    "This notebook requires TensorFlow 2.0 or above."


Using TensorFlow backend.


In [2]:
import gc

In [3]:
import random 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
# UPDATED: import from tensorflow.keras instead of keras
from tensorflow.keras import layers, optimizers, losses, metrics
import gc
import tensorflow_addons as tfa

In [4]:
oneClass = "Consolidation"

In [5]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
from tensorflow.keras.losses import Huber

In [8]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(tf.cast(y_pred,tf.float32) - tf.cast(y_true,tf.float32)), axis=-1))

In [9]:
def construct_network1(dropout=0.08425517073874295, neuronPct=0.1767547775828121, neuronShrink=0.33180474398878285):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat1'))
    model.add(Dense(neuronCount,name='dense1'))
    model.add(Activation('relu',name='relu1'))
    model.add(Dropout(dropout, name='dropout1'))
    model.add(Dense(14, activation='sigmoid',name='midLayer1')) # Output
    weights_path = None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer1')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer1')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [10]:
def construct_network2(dropout=0.15672137551441198, neuronPct=0.2197894476507525, neuronShrink=0.3803316528497302, noisePct=0.282563134185142):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat2'))
    model.add(Dense(neuronCount,name='dense2'))
    model.add(GaussianNoise(noisePct))
    model.add(Activation('relu',name='relu2'))
    model.add(Dropout(dropout, name='dropout2'))
    model.add(Dense(14, activation='sigmoid',name='midLayer2')) # Output
    weights_path = None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    #model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer2')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer2')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [11]:
def construct_network(model1,model2):
    model = Model([model1.input,model2.input], [model1.output,model2.output])
    return model 

In [12]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train"+oneClass, class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev"+oneClass, class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
keras.backend.clear_session()
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)
#print(str(train_steps))
#print(str(train_counts))
#print(str(batch_size))

** compute class weights from training data **
813: 3263
126: 3263
896: 3263
841: 3263
435: 3263
289: 3263
86: 3263
173: 3263
3263: 3263
109: 3263
79: 3263
57: 3263
171: 3263
4: 3263
** class_weights **
[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}]


In [13]:
test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 22433 **


In [14]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
)

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [15]:
def optimize_network(lr):
    gc.collect()
      # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    start_time = time.time()
    model1 = construct_network1()
    model2 = construct_network2()
    
    optimizer = SGD(lr=initial_learning_rate)
    
    alpha = 0.9340456763831478
    gamma = 1.4195808780694898
    model1.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss})

    alpha = 0.7297456293468533
    gamma = 1.2700405014991505
    model2.compile(optimizer=optimizer,loss={'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})
  
    model = construct_network(model1=model1,model2=model2)
    model.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss,'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})

    output_weights_path = os.path.join(output_dir,  str(lr)+"_"+output_weights_name)
    
    print(f"** set output weights path to: {output_weights_path} **")
                  
    
                  
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
  
    model.summary()
  
    callbacks = [
            checkpoint,
            #keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
    ]
    
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=[class_weights,class_weights,class_weights,class_weights,class_weights,class_weights],
            workers=generator_workers,
            shuffle=False,
        )
        
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(lr)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    auprcs = []
    precision = dict()
    recall = dict()
    threshold = dict()
    with open(test_log_path, "w") as f:
        for k in range(6):
            for i in range(len(class_names)):
                 if(class_names[i] == str(oneClass)):
                
                    try:
                        score = roc_auc_score(y[:, i], y_hat[k][:, i])
                        precision[i], recall[i], threshold[i] = precision_recall_curve(y[:, i], y_hat[k][:, i])
                        tmp = auc(recall[i], precision[i])
                        aurocs.append(score)
                        auprcs.append(tmp) 
                    except ValueError:
                        score = 0
               
                    print(f"auroc {str(k)+class_names[i]}: {score}\n")
                    print(f"auprc {str(k)+class_names[i]}: {tmp}\n")
                    f.write(f"auroc {str(k)+class_names[i]}: {score}\n")
                    f.write(f"auprc {str(k)+class_names[i]}: {tmp}\n")
        
        mean_auroc = np.mean(aurocs)
        mean_auprc = float(np.mean(auprcs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}\n")
        f.write(f"mean auprc: {mean_auprc}\n")
        print(f"mean auprc: {mean_auprc}\n")
        
        max_auroc = np.max(aurocs)
        max_auprc = float(np.max(auprcs))
        f.write("-------------------------\n")
        f.write(f"max auroc: {max_auroc}\n")
        print(f"max auroc: {max_auroc}\n")
        f.write(f"max auprc: {max_auprc}\n")
        print(f"max auprc: {max_auprc}\n")
    
    keras.backend.clear_session()
    time_took = time.time() - start_time
    
    return max_auroc, time_took
    


In [16]:
step = np.arange(0.009, 0.0151, 0.001)
maxi = []
for k in np.nditer(step):
    opn, daTime = optimize_network(k)
    print(daTime)
    maxi.append(opn)




** set output weights path to: ./experiments/0.009_weights.h5 **
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
flat2 (Flatten)                 (None, 1536)         0           input_2[0][0]                    
__________________________________________________________________________________________________
flat1 (Flatten)                 (None, 1536)         0           input_1[0][0]                    
___________________________

Epoch 6/11
3253/3263 [============================>.] - ETA: 0s - loss: 4.9825 - leftLayer1_loss: 0.0924 - midLayer1_loss: 1.3982 - rightLayer1_loss: 1.0104 - leftLayer2_loss: 0.0634 - midLayer2_loss: 1.4877 - rightLayer2_loss: 0.9305
Epoch 00006: val_loss improved from 5.04781 to 4.99100, saving model to ./experiments/0.009_weights.h5
3263/3263 [==============================] - 8s 3ms/step - loss: 4.9822 - leftLayer1_loss: 0.0924 - midLayer1_loss: 1.3981 - rightLayer1_loss: 1.0102 - leftLayer2_loss: 0.0633 - midLayer2_loss: 1.4877 - rightLayer2_loss: 0.9304 - val_loss: 4.9910 - val_leftLayer1_loss: 0.0896 - val_midLayer1_loss: 1.3933 - val_rightLayer1_loss: 0.9887 - val_leftLayer2_loss: 0.0773 - val_midLayer2_loss: 1.4020 - val_rightLayer2_loss: 1.0401
Epoch 7/11
3242/3263 [============================>.] - ETA: 0s - loss: 4.9401 - leftLayer1_loss: 0.0880 - midLayer1_loss: 1.3979 - rightLayer1_loss: 0.9874 - leftLayer2_loss: 0.0593 - midLayer2_loss: 1.4836 - rightLayer2_loss: 0.9238


  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 3263 steps, validate for 447 steps
Epoch 1/11
3255/3263 [============================>.] - ETA: 0s - loss: 6.1063 - leftLayer1_loss: 0.1211 - midLayer1_loss: 1.3499 - rightLayer1_loss: 1.6613 - leftLayer2_loss: 0.1222 - midLayer2_loss: 1.4332 - rightLayer2_loss: 1.4185
Epoch 00001: val_loss improved from inf to 5.76616, saving model to ./experiments/0.009999999999999998_weights.h5
3263/3263 [==============================] - 9s 3ms/step - loss: 6.1050 - leftLayer1_loss: 0.1211 - midLayer1_loss: 1.3498 - rightLayer1_loss: 1.6607 - leftLayer2_loss: 0.1222 - midLayer2_loss: 1.4333 - rightLayer2_loss: 1.4177 - val_loss: 5.7662 - val_leftLayer1_loss: 0.1171 - val_midLayer1_loss: 1.3464 - val_rightLayer1_loss: 1.4710 - val_leftLayer2_loss: 0.1138 - val_midLayer2_loss: 1.3758 - val_rightLayer2_loss: 1.3

3241/3263 [============================>.] - ETA: 0s - loss: 4.7579 - leftLayer1_loss: 0.0721 - midLayer1_loss: 1.3509 - rightLayer1_loss: 0.9412 - leftLayer2_loss: 0.0498 - midLayer2_loss: 1.4327 - rightLayer2_loss: 0.9111
Epoch 00011: val_loss improved from 4.78741 to 4.77061, saving model to ./experiments/0.009999999999999998_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.009999999999999998_test.log **
auroc 0Consolidation: 0.6339749769030891

auprc 0Consolidation: 0.055041709609329074

auroc 1Consolidation: 0.4993421248535217

auprc 1Consolidation: 0.0395799723976445

auroc 2Consolidation: 0.5351717248268972

auprc 2Consolidation: 0.043443787591666186

auroc 3Consolidation: 0.4260137874982995

auprc 3Consolidation: 0.033870729578634455

auroc 4Consolidation: 0.4508266913293214

auprc 4Consolidation: 0.0366910763465656

auroc 5Consolidation: 0.4482993628230332

auprc 5Consolidation: 0.03585022907392379

mean auroc: 0.4989381113

Epoch 4/11
3246/3263 [============================>.] - ETA: 0s - loss: 5.0413 - leftLayer1_loss: 0.1034 - midLayer1_loss: 1.3591 - rightLayer1_loss: 1.1205 - leftLayer2_loss: 0.0795 - midLayer2_loss: 1.4194 - rightLayer2_loss: 0.9595
Epoch 00004: val_loss improved from 5.23806 to 5.10745, saving model to ./experiments/0.010999999999999998_weights.h5
3263/3263 [==============================] - 8s 3ms/step - loss: 5.0406 - leftLayer1_loss: 0.1034 - midLayer1_loss: 1.3590 - rightLayer1_loss: 1.1201 - leftLayer2_loss: 0.0795 - midLayer2_loss: 1.4195 - rightLayer2_loss: 0.9592 - val_loss: 5.1075 - val_leftLayer1_loss: 0.1006 - val_midLayer1_loss: 1.3531 - val_rightLayer1_loss: 1.0785 - val_leftLayer2_loss: 0.0905 - val_midLayer2_loss: 1.3841 - val_rightLayer2_loss: 1.1007
Epoch 5/11
3261/3263 [============================>.] - ETA: 0s - loss: 4.9484 - leftLayer1_loss: 0.0983 - midLayer1_loss: 1.3577 - rightLayer1_loss: 1.0608 - leftLayer2_loss: 0.0719 - midLayer2_loss: 1.4176 - rightLayer

...

3261/3263 [============================>.] - ETA: 0s - loss: 4.8775 - leftLayer1_loss: 0.0770 - midLayer1_loss: 1.4237 - rightLayer1_loss: 0.9527 - leftLayer2_loss: 0.0496 - midLayer2_loss: 1.4604 - rightLayer2_loss: 0.9141
Epoch 00011: val_loss improved from 4.89644 to 4.87799, saving model to ./experiments/0.011999999999999997_weights.h5
22433/22433 [==============================] - 27s 1ms/step
** write log to ./experiments/0.011999999999999997_test.log **
auroc 0Consolidation: 0.718296850237236

auprc 0Consolidation: 0.09577556858142627

auroc 1Consolidation: 0.46003830574257226

auprc 1Consolidation: 0.03780286927934735

auroc 2Consolidation: 0.6355550498595501

auprc 2Consolidation: 0.07385868954836547

auroc 3Consolidation: 0.4178723331996272

auprc 3Consolidation: 0.03314810996911039

auroc 4Consolidation: 0.5354802269618166

auprc 4Consolidation: 0.043573470729683046

auroc 5Consolidation: 0.4095485412697569

auprc 5Consolidation: 0.03252563955530309

mean auroc: 0.5294652178

Epoch 4/11
3254/3263 [============================>.] - ETA: 0s - loss: 5.2018 - leftLayer1_loss: 0.1005 - midLayer1_loss: 1.3557 - rightLayer1_loss: 1.1030 - leftLayer2_loss: 0.0758 - midLayer2_loss: 1.6092 - rightLayer2_loss: 0.9577
Epoch 00004: val_loss improved from 5.25888 to 5.13296, saving model to ./experiments/0.012999999999999996_weights.h5
3263/3263 [==============================] - 8s 2ms/step - loss: 5.2012 - leftLayer1_loss: 0.1005 - midLayer1_loss: 1.3556 - rightLayer1_loss: 1.1027 - leftLayer2_loss: 0.0758 - midLayer2_loss: 1.6090 - rightLayer2_loss: 0.9577 - val_loss: 5.1330 - val_leftLayer1_loss: 0.0974 - val_midLayer1_loss: 1.3438 - val_rightLayer1_loss: 1.0619 - val_leftLayer2_loss: 0.0876 - val_midLayer2_loss: 1.4716 - val_rightLayer2_loss: 1.0707
Epoch 5/11
3250/3263 [============================>.] - ETA: 0s - loss: 5.1170 - leftLayer1_loss: 0.0953 - midLayer1_loss: 1.3549 - rightLayer1_loss: 1.0471 - leftLayer2_loss: 0.0684 - midLayer2_loss: 1.6107 - rightLayer

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 3263 steps, validate for 447 steps
Epoch 1/11
3248/3263 [============================>.] - ETA: 0s - loss: 6.5622 - leftLayer1_loss: 0.1256 - midLayer1_loss: 1.4954 - rightLayer1_loss: 1.6633 - leftLayer2_loss: 0.1160 - midLayer2_loss: 1.6737 - rightLayer2_loss: 1.4881
Epoch 00001: val_loss improved from inf to 6.11253, saving model to ./experiments/0.013999999999999995_weights.h5
3263/3263 [==============================] - 9s 3ms/step - loss: 6.5596 - leftLayer1_loss: 0.1256 - midLayer1_loss: 1.4953 - rightLayer1_loss: 1.6624 - leftLayer2_loss: 0.1160 - midLayer2_loss: 1.6737 - rightLayer2_loss: 1.4866 - val_loss: 6.1125 - val_leftLayer1_loss: 0.1214 - val_midLayer1_loss: 1.4878 - val_rightLayer1_loss: 1.4833 - val_leftLayer2_loss: 0.1137 - val_midLayer2_loss: 1.5031 - val_rightLayer2_loss: 1.4

3251/3263 [============================>.] - ETA: 0s - loss: 5.1529 - leftLayer1_loss: 0.0750 - midLayer1_loss: 1.4961 - rightLayer1_loss: 0.9421 - leftLayer2_loss: 0.0503 - midLayer2_loss: 1.6768 - rightLayer2_loss: 0.9125
Epoch 00011: val_loss improved from 5.08749 to 5.06923, saving model to ./experiments/0.013999999999999995_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.013999999999999995_test.log **
auroc 0Consolidation: 0.5915281630506645

auprc 0Consolidation: 0.05283388820723718

auroc 1Consolidation: 0.5365463243166341

auprc 1Consolidation: 0.044960769724129754

auroc 2Consolidation: 0.5367575634963129

auprc 2Consolidation: 0.04832880427102941

auroc 3Consolidation: 0.36154898092361565

auprc 3Consolidation: 0.030030912011885506

auroc 4Consolidation: 0.5551708178826823

auprc 4Consolidation: 0.047335095961546246

auroc 5Consolidation: 0.39620473526084277

auprc 5Consolidation: 0.03217400580151656

mean auroc: 0.496292

Epoch 4/11
3252/3263 [============================>.] - ETA: 0s - loss: 5.0289 - leftLayer1_loss: 0.0990 - midLayer1_loss: 1.4107 - rightLayer1_loss: 1.0867 - leftLayer2_loss: 0.0786 - midLayer2_loss: 1.3949 - rightLayer2_loss: 0.9591
Epoch 00004: val_loss improved from 5.20087 to 5.08078, saving model to ./experiments/0.014999999999999994_weights.h5
3263/3263 [==============================] - 8s 3ms/step - loss: 5.0284 - leftLayer1_loss: 0.0989 - midLayer1_loss: 1.4106 - rightLayer1_loss: 1.0865 - leftLayer2_loss: 0.0786 - midLayer2_loss: 1.3948 - rightLayer2_loss: 0.9589 - val_loss: 5.0808 - val_leftLayer1_loss: 0.0959 - val_midLayer1_loss: 1.4071 - val_rightLayer1_loss: 1.0493 - val_leftLayer2_loss: 0.0899 - val_midLayer2_loss: 1.3395 - val_rightLayer2_loss: 1.0991
Epoch 5/11
3244/3263 [============================>.] - ETA: 0s - loss: 4.9527 - leftLayer1_loss: 0.0938 - midLayer1_loss: 1.4118 - rightLayer1_loss: 1.0358 - leftLayer2_loss: 0.0712 - midLayer2_loss: 1.3974 - rightLayer

In [17]:
print(np.max(maxi))

0.718296850237236
